# Tiles with Borders

**Contents**

* [Introduction](#introduction)
* [Tile Properties](#Tile-Properties)
* [Movement into Tiles](#Movement-into-Tiles)
* [Coordination](#Coordination)
  * [A Single, Non-distributed System](#A-Single,-Non-distributed-System)
  * [A Distributed System](#A-Distributed-System)
* [Data](#Data)
* [Footnotes](#Footnotes)

## Introduction

We're exploring tiles of width $2^n$, where $n \geqslant 1$ (since tiles of width $2^0$ are just single cells). Powers of 2 are both practical for scaling purposes as well as making the math easier.  With cells making up tiles, we'll also look at collections of tiles; these too will have widths of powers of two, namely $2^m$. We'll spend most of our time working with the case $m=2$, since that's enough tiles two explore all possible interactions. Here's a visual of $m = 0$, $m = 1$, and $m = 2$:

[![Example 4x4 tiles](files/images/4x4-tile-examples-thumb.png)](files/images/4x4-tile-examples.png)

The example above shows tiles with borders; furthermore, it highlights where those borders may have interaction with other tiles, and what type of interactions. In particular:

* <font color="#C27BA0" size="10px">&#9632;</font> Cells with this colour are conflcit-free, contention-free zone: they will never have shared data for time steps  $t + 1$.
* <font color="#6D9EEB" size="10px">&#9632;</font> Cells with this colour are part of a tile's outer border. Since they are part of the outer border, with no overlap with cells of other tiles, they can only be considered temporarily conflict-free. This is due to the fact that the tiles could be moved or other tiles could be placed against their borders. As such, at every time step, we should check if a new tile has been placed adjacent; if not, we may continue to regard it as conflict-free for time steps  $t + 1$.
* <font color="#3C78D8" size="10px">&#9632;</font> Dark blue cells are outer border cells of a tile that are adjacent to other tiles's outer border cells and could have shared data between two cells of two different tiles for time steps $t + 1$.
* <font color="#8E7CC3" size="10px">&#9632;</font> Light purple cells are non-outer border cells of two adjacent tiless' non-outer border cells and could have shared data between two cells of two different tiles for time steps $t + 1$.
* <font color="#674EA7" size="10px">&#9632;</font> Dark purple cells are non-outer border cells of four adjacent tiles' non-outer border cells and could have shared data between fours cells of four different tiles for time steps $t + 1$.

The idea here is that if we split up an agent-based world into tiles, how much processing in those tiles can be done independently of the others, and how much will need to be coordinated? If a model is running across a collection of tiles and time $t_0$ is the initial state, then at $t_1$, the most any tile's agents could make it into another tile would be around the perimater of the tile to a depth of one cell. It is in this case (where agent data has moved from one tile to another) that coordination is required between the work being done on tiles; conversely,  agents who remain in a given tile don't need to have their work coordinated.


## Tile Properties

In general, each tile has the following properties:

* Cell dimentions: $2^n$ by $2^n$
* Total number of cells: $2^n \times 2^n$
* Number of inner cells: $(2^n - 2) \times (2^n - 2)$
* Number of border cells: $2^n \times 2^n - ((2^n - 2) \times (2^n - 2))$

Collections of tiles follow the same rules, and as such, have the same properies:

* Tile set dimentions: $2^m$ by $2^m$
* Total number of tiles in a set: $2^m \times 2^m$
* Number of inner tiles: $(2^m - 2) \times (2^m - 2)$
* Number of border tiles: $2^m \times 2^m - ((2^m - 2) \times (2^m - 2))$

We can write some functions that define these for us:

In [1]:
to_lisp()$


Type (to-maxima) to restart, ($quit) to quit Maxima.


In [2]:
;; Decided not to use this next one
;; (defun power2p (n)
;;   (if (eql 0 (and (/= n 0) (logand n (- n 1))))
;;     t
;;     nil))
    
(defun total-count (n)
  (* (expt 2 n) (expt 2 n)))
  
(defun inner-count (n)
  (* (- (expt 2 n) 2) 
     (- (expt 2 n) 2)))

;; This is less efficient than the new one, below
;; (defun border-count (n)
;;  (- (total-count n) (inner-count n)))

(defun border-count (n)
  (+ (* (- (expt 2 n) 2) 4) 4))

(defstruct props power width total-count inner-count border-count)

(defun get-props (n)
  (make-props
    :power n
    :width (expt 2 n)
    :total-count (total-count n)
    :inner-count (inner-count n)
    :border-count (border-count n)))

TOTAL-COUNT

INNER-COUNT

BORDER-COUNT

PROPS

GET-PROPS

So, for the trivial case:

In [3]:
(get-props 0)

#S(PROPS :POWER 0 :WIDTH 1 :TOTAL-COUNT 1 :INNER-COUNT 1 :BORDER-COUNT 0)

The case of $n = 1$ isn't all that interesting from a practical perspective (since no cells are conflict free), but here are its properties:

In [4]:
(get-props 1)

#S(PROPS :POWER 1 :WIDTH 2 :TOTAL-COUNT 4 :INNER-COUNT 0 :BORDER-COUNT 4)

And a visual for it in single tile, four-tile, and 16-tile arrangements:

[![Example 2x2 tiles](files/images/2x2-tile-examples-thumb.png)](files/images/2x2-tile-examples.png)

We saw above, when defining the different cell types, an example of $n=2$, but let's look again:

In [5]:
(get-props 2)

#S(PROPS :POWER 2 :WIDTH 4 :TOTAL-COUNT 16 :INNER-COUNT 4 :BORDER-COUNT 12)

Again, the visual:

[![Example 4x4 tiles](files/images/4x4-tile-examples-thumb.png)](files/images/4x4-tile-examples.png)

Properties for $n = 3$:

In [6]:
(get-props 3)

#S(PROPS :POWER 3 :WIDTH 8 :TOTAL-COUNT 64 :INNER-COUNT 36 :BORDER-COUNT 28)

And we have this set of tiles (one tile, four tiles, and 16 tiles):

[![Example 8x8 tiles](files/images/8x8-tile-examples-thumb.png)](files/images/8x8-tile-examples.png)

Properties for a few more, with no corresponding visuals:

In [7]:
(get-props 4)
(get-props 5)
(get-props 6)

#S(PROPS :POWER 4 :WIDTH 16 :TOTAL-COUNT 256 :INNER-COUNT 196 :BORDER-COUNT 60)

#S(PROPS
   :POWER 5
   :WIDTH 32
   :TOTAL-COUNT 1024
   :INNER-COUNT 900
   :BORDER-COUNT 124)

#S(PROPS
   :POWER 6
   :WIDTH 64
   :TOTAL-COUNT 4096
   :INNER-COUNT 3844
   :BORDER-COUNT 252)

These examples of 4x4, 8X8, and 16x16 tiles will be useful for exploration, but in all likelihood real-world examples will be tiles of thousands of cells (think roaring bitmap sizes).

## Movement into Tiles

For this next section, we'll look at an example agent-based model running Conway's Game of Life. Rather, we'll look at a "glider" that starts in one tile, and progresses until it begins to move into another tile. Then we'll explore possible solutions to the distributed coordination problem.

Here's a summary of the sequence we'll be looking at
* $t_0$: glider starts where none of its agent cells can impact cells in other tiles
* $t_1$ through $t_4$: glider moves downward, toward another tile, until nearly all of its tiles are in one or more border zones
* $t_5$: one cell of the glider moves into a different tile

Here's the glider at $t_0$:

[![4x4 Glider t=0](files/images/4x4-glider-t0-thumb.png)](files/images/4x4-glider-t0.png)

But let's shade the other tiles, to indicate that we don't know what might be in them:

[![4x4 Glider t=0](files/images/4x4-glider-t0-shaded-thumb.png)](files/images/4x4-glider-t0-shaded.png)

The next four time steps:

[![4x4 Glider t=1 to t=4](files/images/4x4-glider-t1-t4-shaded-thumb.png)](files/images/4x4-glider-t1-t4-shaded.png)

Of particular note in the above:

* At $t_1$ and $t_2$, only one other tile may be impacted, if that tile has neighboring agents adjacent to the cells of our glider.
* At $t_3$, the glider has moved such that two other tiles may be impacted by its agents (or may be impacted by agents in either or both of the other tiles).
* At $t_4$, the glider has moved an agent into a cell that can impact or be impacted by data in all four of the tiles. These sorts of zone will, of course, require the most coordination.

Then, at $t_5$, the glider crosses from one tile into another:

[![4x4 Glider t=5](files/images/4x4-glider-t5-shaded-thumb.png)](files/images/4x4-glider-t5-shaded.png)

## Coordination

In order for the tiles to be able to determine whether one of their agents is impacted by something going on in another tile, some form of coordination needs to take place. If we assume a fixed number of tiles in a fixed position, then the only situations where coordination will need to take place is if a tile has an agent in one of the border cells and is in proxity to an agent in an adjacent tile.

The rules for the Game of Life define the neighborhood of an agent to be all surrounding eight cells, so the adjacency we care about for this agent-based model is only ever one cell away, though in all directions.

The _degree_ to which coordinated needs to be performed depends upon the _type_ of border cell. The term _degree_ here means with how many other tiles will there needs to be coordination.

So what are the possible forms this coordination could take?

### A Single, Non-distributed System

* Shared memory with mutex's<sup>[1](#lockingimpl)</sup>
  * Any time a tile needs to put an agent in a cell of its border zone, it performs a lock; 
  * This is needed, since other tiles might have an agent leaving their tile and moving to another tile.
  * Reading will also need to be locked and unlocked.
* Share memory by channels<sup>[2](#cspimpl)</sup>
* Communicating with immutable data<sup>[3](#immutimpl)</sup>
  
### A Distributed System

* We could store data in distributed database clusters
  * This would definitely be very slow compared to in-memory solutions.
  * However, for models with data that is too large to fit in a single system, such benefits could outweigh the performance detriments.
* We could have a naïve messaging system: 
  * Generate a message every time a cell changes (e.g., an agent enters or leaves a cell).
  * Neighboring tiles would only need to subscribe to changes in the border cells of the adjacent tiles.
  * Tiles would only need to pay attention to these change messages if there was an agent in a neighboring cell of their own border zone.
* A more sophisticated messaging system:
  * One that optimizes/minimizes the amount of messages generated, sending larger chunks of state data.
  * This would reduce messaging chatter as well as give receving code more to process at one go.
  
## Data

In our communicating notes above, one of the things we glossed over is the manner in which each tile tracks its data, and how that data is represented. So what are the basic requirements?

* A data structure that can set and get data representing points in a 2D plane.
* The ability to quickly query this data structure for all points in a border zone.
* The ability to quickly query this data structure for all points in a neighborhood.<sup>[4](#neighborhooddef)</sup>
* The ability to index all cells (or agents in cells) so that their metadata (stored elsewhere) may be easily retrieved<sup>[5](#metadata)</sup>

Possible data structures to use:
* Arrays
* Quadtrees <sup>[6](#quadtree)</sup>
* R-trees <sup>[7](#rtree)</sup>
  * dynamic Hilbert R-tree <sup>[8](#dynhilrtree)</sup>
* Bitmaps
  * Roaring Bitmaps <sup>[9](#rbitmaps)</sup>

How can our requirements above be satisfied by each of these data structures?

More TBD ...

## Footnotes

<a name="lockingimpl">1</a>: It might be a good idea to have a locking implementation, just to having something to compare to. See https://gobyexample.com/mutexes and  https://stackoverflow.com/questions/55196635/when-to-use-mutexes.

<a name="cspimpl">2</a>: If we do a locking example, we should also do a CSP example. See https://blog.golang.org/share-memory-by-communicating and https://golang.org/doc/codewalk/sharemem/.

<a name="immutimpl">3</a>: See, for example, https://github.com/tobgu/peds/. Immutable data structures are not supported natively in Go; would need to see some compelling benchmarks and a good use case to pursue this.

<a name="neighborhooddef">4</a>: For the Game of Life, as previously mentioned, the naïve definition of neighboring cells is used: those eight cells immediately adjacent to (surrounding) a given cell. However, we need to be flexible here, due to the fact that different agent-based models will have potentially wildly varying definitions of neighborhood. Consider the possibility that models could want the nearest $n$ cells, an actual radius (for discrete cells, this would have to be approximated, but it wouldn't produce the same results as nearst $n$ cells), nearst $n$ agents, all agents within a given radius, agents or cells within a function of distance (as opposed to scalar values for distance), or even proximity over time. Note, too, that the definition of a border zone is ultimately dependent upon the definition of a neighborhood: for all cells that border another tile, the definition of neighborhood needs to be applied, and any cells qualifying as neighbors according to that definition, comprise the border zone.

<a name="metadata">5</a>: We don't actually want to store metadata in the data structure representing the cells/agents. The presence of an agent in a cell will potentially be highly dynamic, changing a great deal and in all likelihood can be representing with a single on/off bit, so could be very compact. Non-state metadata, however, could potentially be very verbose and is typically also static in nature; thus, it would be a waste to store this in a data structure that it optimized for small, rapidly changing bits.

<a name="quadtree">6</a>: Quadtrees will probably not be much use in our case. If, however, we eventually care about varying scales in large tiles, it would become a pertinent option.

<a name="rtree">7</a>: R-trees could be useful for grouping agents in agent-based models that have more sophisticated definitions of neighborhood than the Game of Life model does.

<a name="dynhilrtree">8</a>: See https://en.wikipedia.org/wiki/Hilbert_R-tree#Dynamic_Hilbert_R-trees.

<a name="rbitmaps">9</a>: See https://www.roaringbitmap.org/ and https://github.com/RoaringBitmap/roaring.